In [1]:
import TextualData.TextualData
import LSTM.lstm as lstm
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
import time
import spacy

In [2]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
nlp = spacy.load('en')

In [4]:
text = TextualData.TextualData.TextualData(path='data/full_shak_eng.txt')

In [16]:
doc = nlp(text.full_text[:300000])
allpos = set([tok.pos_ for tok in doc])
pos_to_ix = {}
ix_to_pos = {}
ix=0
for pos in allpos:
    pos_to_ix[pos]=ix
    ix_to_pos[ix]=pos
    ix+=1
mylen = 100000
begin = 0
end = mylen
full_pos = []
while begin < len(text.full_text):
    mystr = text.full_text[begin:end]
    begin = end
    end = min(end+mylen, len(text.full_text))
    doc = nlp(mystr)
    pos_by_char = []
    for tok in doc:
        pos_by_char.append([pos_to_ix[tok.pos_] for l in tok.text_with_ws])
    pos_by_char = np.array([item for sublist in pos_by_char for item in sublist])
    full_pos.append(pos_by_char)
    print begin, end, len(pos_by_char)

100000 200000 100000
200000 300000 100000
300000 400000 100000
400000 500000 100000
500000 600000 100000
600000 700000 100000
700000 800000 100000
800000 900000 100000
900000 1000000 100000
1000000 1100000 100000
1100000 1200000 100000
1200000 1300000 100000
1300000 1400000 100000
1400000 1500000 100000
1500000 1600000 100000
1600000 1700000 100000
1700000 1800000 100000
1800000 1900000 100000
1900000 2000000 100000
2000000 2100000 100000
2100000 2147990 100000
2147990 2147990 47990


In [17]:
full_pos = [item for sublist in full_pos for item in sublist]

In [18]:
print len(full_pos)
print len(text.full_text)

2147990
2147991


In [ ]:
start = 0
string_len = 10
batch_size = 1
stride = 1

if start + string_len * batch_size >= text.train_len:
    raise IndexError('Not enough text to return this batch')
batch = []
st = start
for i in range(batch_size):
    batch.append(text.string_to_tensor(text.train_text[st:st + string_len]))
    st += string_len - 1
# batch = [self.string_to_tensor(self.random_string_fixed_size(string_len)) for i in range(batch_size)]
targets = [torch.cat([text.onehot_to_class(b[0]) for b in batch[bn][stride:]]) for bn in range(batch_size)]
batch = torch.cat(batch, dim=1)[:string_len - stride]
batch = [[batch[:, i], targets[i]] for i in range(batch_size)]